# TRABAJO PRÁCTICO FINAL

##Instalar e importar librerías necesarias

In [1]:
!pip install kaggle
!pip install PyPDF2
!pip install wikidataintegrator sparqlwrapper
!pip install langchain
!pip install sentence-transformers
!pip install chromadb
!pip install decouple

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for 

In [170]:
# Google Colab
from google.colab import files

# Zip
import zipfile
import os

# PDF
from PyPDF2 import PdfReader

# Regular Expressions
import re

# Data manipulation
import pandas as pd
import numpy as np

# Wikidata
from wikidataintegrator import wdi_core, wdi_login

# HTTP requests
import requests

# Text processing
from langchain.text_splitter import RecursiveCharacterTextSplitter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sentence_transformers import SentenceTransformer
from langchain_core.documents.base import Document

# Database
import chromadb

# Transformers
from transformers import MarianMTModel, MarianTokenizer

# JSON handling
import json

# Template rendering
from jinja2 import Template

# Type hints
from typing import Any, Dict, List

# Miscellaneous
import warnings
warnings.filterwarnings("ignore")

## Credenciales para APIs

Ingresá acá las credenciales de Wikidata, la API key de Hugging Face y subí el archivo kaggle.json para poder usar las herramientas de este trabajo.

In [171]:
# Configurar la conexión a Wikidata
wikidata_user = ''
wikidata_password = ''

# Tu clave API de Hugging Face
api_key = ''

# Selecciona tu archivo kaggle.json
uploaded = files.upload()

Saving kaggle.json to kaggle.json


## 1. Fuentes de datos

A continuación se preparan las fuentes de datos para dar contexto al LLM.

### 1.1. Texto
Para la fuente de texto elegí los siete libros de Harry Potter en formato PDF. Para poder procesarlo posteriormente, primero se descargaran los libros de un dataset de Kaggle, se descomprimirá el archivo .zip y se leerán los PDFs con PyPDF2.

In [172]:
#Mover el Archivo Kaggle a la Ruta Adecuada
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Descargar libros con Kaggle API
!kaggle datasets download -d hinepo/harry-potter-books-in-pdf-1-7

# Ruta del archivo zip
zip_file_path = 'harry-potter-books-in-pdf-1-7.zip'

# Descomprimir el archivo zip en la carpeta actual
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

harry-potter-books-in-pdf-1-7.zip: Skipping, found more recently modified local copy (use --force to force download)


In [173]:
def extract_text_from_pdfs(pdf_folder):
    pdf_texts = []

    for pdf_file in os.listdir(pdf_folder):
        if pdf_file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, pdf_file)
            with open(pdf_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                text = ""
                for page_num in range(len(pdf_reader.pages)):
                    page_text = pdf_reader.pages[page_num].extract_text()
                    # Reemplazar saltos de línea simples con espacios y conservar solo dobles
                    cleaned_text = "\n\n".join(para.strip() for para in page_text.split('\n\n'))
                    text += cleaned_text
            pdf_texts.append(text)

    return pdf_texts

In [174]:
# Ruta de la carpeta con los archivos PDF
pdf_folder_path = 'HP books'

# Extraer texto de los PDFs
pdf_texts = extract_text_from_pdfs(pdf_folder_path)

# Mostrar el texto extraído de un ejemplo de PDF
print(pdf_texts[0][:500])  # Muestra los primeros 500 caracteres del primer PDF

P a g e  | 2 Harry Potter and the Half Blood Prince – J.K. Rowling  
 
THE OTHER MINISTER 
It was nearing midnight and the Prime Minister was 
sitting alone in his office, reading a long memo that 
was slipping through his brain without leaving the 
slightest trace of meaning behind. He was waiting for 
a call from the President of a far distant country, and 
between wondering when the wretched man would 
telephone, and trying to suppress unpleasant 
memories of what had been a very long, tiring


In [175]:
def clean_newlines_before_words(text):
    # Utilizamos una expresión regular para buscar saltos de línea seguidos de una palabra
    cleaned_text = re.sub(r'\n(?=\w)', ' ', text)
    return cleaned_text

> Se limpian los saltos de línea "ficticios" es decir, que existen sólo porque se llegó al extremo de la hoja del pdf y no porque tenga un sentido semántico. Esto es porque el splitter de texto de langchain utiliza los saltos de línea como guía para dividir el texto.

In [176]:
# Limpiar saltos de línea pegados a palabras en todos los PDFs
cleaned_texts = [clean_newlines_before_words(pdf_text) for pdf_text in pdf_texts]

# Mostrar el texto limpio de un ejemplo de PDF
print(cleaned_texts[0][:500])  # Muestra los primeros 500 caracteres del primer texto limpio

P a g e  | 2 Harry Potter and the Half Blood Prince – J.K. Rowling  
  THE OTHER MINISTER  It was nearing midnight and the Prime Minister was  sitting alone in his office, reading a long memo that  was slipping through his brain without leaving the  slightest trace of meaning behind. He was waiting for  a call from the President of a far distant country, and  between wondering when the wretched man would  telephone, and trying to suppress unpleasant  memories of what had been a very long, tiring


### 1.2. Datos numéricos tabulares

Para los datos numéricos tabulares utilicé un pequeño dataset con datos sobre las novelas físicas, es decir, unidades vendidas, fecha de lanzamiento, etc... El dataset está disponible también en Kaggle.

In [14]:
# Descargar CSV con Kaggle API
!kaggle datasets download -d adnananam/harry-potter-dataset

  0% 0.00/596 [00:00<?, ?B/s]
100% 596/596 [00:00<00:00, 1.67MB/s]


In [15]:
# Ruta del archivo zip
zip_file_path = 'harry-potter-dataset.zip'

# Descomprimir el archivo zip en la carpeta actual
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall()

In [16]:
# Ruta del archivo CSV
csv_file_path = 'Harry_Potter_Scraped_Dataset.csv'

# Cargar el archivo CSV en un DataFrame de pandas
df_harry_potter = pd.read_csv(csv_file_path)

# Mostrar las primeras filas del DataFrame para verificar la importación
df_harry_potter.head(7)

,Book Name,Copies Sold in UK,Copies Sold WorldWide,Publish Date,Publish Month,Publish Year,Pages,Words,Audiobook,Total Awards Won
0,Harry Potter and the Sorcerer’s Stone,4.2,120,26,June,1997,223,"76,944",9 hrs and 33 mins,8
1,Harry Potter and the Chamber of Secrets,3.5,77,2,July,1998,251,"85,141",11 hrs and 5 mins,5
2,Harry Potter and the Prisoner of Azkaban,3.4,65,8,July,1999,317,"107,253",13 hrs and 10 mins,3
3,Harry Potter and the Goblet of Fire,3.6,65,8,July,2000,636,"190,637",22 hrs and 17 mins,1
4,Harry Potter and the Order of the Phoenix,4.2,65,21,June,2003,766,"257,045",30 hrs and 18 mins,1
5,Harry Potter and the Half-blood Prince,4.1,65,16,July,2005,607,"168,923",21 hrs and 27 mins,1
6,Harry Potter and the Deathly Hallows,4.5,65,21,July,2007,607,"198,227",24 hrs,0


> Para brindar contexto con esta tabla, se define una función que la transforma en texto. De haber dispuesto de más tiempo hubiera convenido detectar a qué libro hace referencia la pregunta y extraer sólo la información deseada. Dadas las circunstancias y al ser tan pequeño el dataset opté por esta alternativa.

In [126]:
# Función para convertir el DataFrame en un string
def dataframe_to_string(df):
    result = ""
    for index, row in df.iterrows():
        result += f"Book Name: {row['Book Name']}\n"
        result += f"Copies Sold in UK: {row['Copies Sold in UK']} millons\n"
        result += f"Copies Sold WorldWide: {row['Copies Sold WorldWide']} millons\n"
        result += f"Publish Date: {row['Publish Date']} {row['Publish Month']} {row['Publish Year']}\n"
        result += f"Pages: {row['Pages']}\n"
        result += f"Words: {row['Words']}\n"
        result += f"Audiobook: {row['Audiobook']}\n"
        result += f"Total Awards Won: {row['Total Awards Won']}\n"
        result += "\n"

    return result

# Convertir el DataFrame en un string
result_string = dataframe_to_string(df_harry_potter)

# Imprimir el resultado
print(result_string)

Book Name: Harry Potter and the Sorcerer’s Stone
Copies Sold in UK: 4.2 millons
Copies Sold WorldWide: 120 millons
Publish Date: 26 June 1997
Pages: 223
Words: 76,944
Audiobook: 9 hrs and 33 mins
Total Awards Won: 8

Book Name: Harry Potter and the Chamber of Secrets
Copies Sold in UK: 3.5 millons
Copies Sold WorldWide: 77 millons
Publish Date: 2 July 1998
Pages: 251
Words: 85,141
Audiobook: 11 hrs and 5 mins
Total Awards Won: 5

Book Name: Harry Potter and the Prisoner of Azkaban
Copies Sold in UK: 3.4 millons
Copies Sold WorldWide: 65 millons
Publish Date: 8 July 1999
Pages: 317
Words: 107,253
Audiobook: 13 hrs and 10 mins
Total Awards Won: 3

Book Name: Harry Potter and the Goblet of Fire
Copies Sold in UK: 3.6 millons
Copies Sold WorldWide: 65 millons
Publish Date: 8 July 2000
Pages: 636
Words: 190,637
Audiobook: 22 hrs and 17 mins
Total Awards Won: 1

Book Name: Harry Potter and the Order of the Phoenix
Copies Sold in UK: 4.2 millons
Copies Sold WorldWide: 65 millons
Publish Date:

### 1.3. Base de datos de Grafos

Para la base de datos de grafos elegí a Wikidata para hacer consultas online. Esta fuente se utilizará únicamente si se hacen preguntas sobre datos de los personajes. Para esto se hace una consulta para obtener el ID de sujeto de la etiqueta del personaje, y luego se las etiquetas de los objetos para un conjunto de predicados preseleccionados.

In [59]:
def obtener_valores_por_sujeto(label_sujeto):
    # Iniciar sesión en Wikidata
    login_instance = wdi_login.WDLogin(wikidata_user, wikidata_password)

    # Probar que la conexión funciona con una consulta SPARQL
    sparql_query = f'''
    SELECT ?sujeto ?sujetoLabel
    WHERE {{
      ?sujeto rdfs:label "{label_sujeto}"@en.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    '''

    # Ejecutar la consulta SPARQL
    result = wdi_core.WDItemEngine.execute_sparql_query(sparql_query, as_dataframe=True)

    # Extracción del ID
    try:
        valor_sujeto = result['sujeto'].str.split('/').str[-1]
    except (KeyError, AttributeError, IndexError):
        # Manejar el error y retornar un string vacío en caso de cualquier excepción esperada
        return ''

    # Imprimir el resultado
    sujeto_id = valor_sujeto.values[0]

    description = f'{label_sujeto}  :\n'

    propiedades = ["P21", "P27", "P1559", "P1477", "P735", "P569", "P19", "P570", "P1196", "P509",
                   "P157", "P22", "P25", "P3373", "P3342", "P106", "P108", "P39", "P551", "P463"]

    for propiedad_id in propiedades:
        # Construir la consulta SPARQL para obtener los valores por sujeto y propiedad
        consulta_sparql = f"""
        SELECT ?property ?propertyLabel ?value ?valueLabel
        WHERE {{
          wd:{sujeto_id} wdt:{propiedad_id} ?value.
          ?property wikibase:directClaim wdt:{propiedad_id}.
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        """

        # URL del punto de consulta SPARQL de Wikidata
        endpoint_url = "https://query.wikidata.org/sparql"

        # Encabezados HTTP necesarios para la consulta a Wikidata
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0',
            'Accept': 'application/json'
        }

        # Parámetros de la consulta SPARQL
        params = {
            'query': consulta_sparql,
            'format': 'json'
        }

        # Realizar la solicitud HTTP GET
        respuesta = requests.get(endpoint_url, headers=headers, params=params)

        # Verificar si la solicitud fue exitosa (código de estado 200)
        if respuesta.status_code == 200:
            # Convertir la respuesta a formato JSON
            resultados = respuesta.json()

            # Obtener los valores asociados a la propiedad
            valores = [(item['propertyLabel']['value'], item['valueLabel']['value']) for item in resultados['results']['bindings']]

            for propiedad, valor in valores:
                description += f'{propiedad}: {valor} \n'
        else:
            description = ''

    return description


Ejemplo:

In [60]:
# ID de Albus Dumbledore en Wikidata
sujeto_label_ejemplo = "Albus Dumbledore"

# Obtener y mostrar los valores asociados a la propiedad
propiedades_personaje = obtener_valores_por_sujeto(sujeto_label_ejemplo)

print(propiedades_personaje)

Albus Dumbledore  :
sex or gender: male 
country of citizenship: United Kingdom 
name in native language: Albus Percival Wulfric Brian Dumbledore 
birth name: Albus Percival Wulfric Brian Dumbledore 
given name: Percival 
given name: Brian 
given name: Albus 
given name: Wulfric 
date of birth: 1881-01-01T00:00:00Z 
place of birth: Mould-on-the-Wold 
date of death: 1997-06-30T00:00:00Z 
manner of death: homicide 
cause of death: Avada Kedavra 
killed by: Severus Snape 
father: Percival Dumbledore 
mother: Kendra Dumbledore 
sibling: Aberforth Dumbledore 
sibling: Ariana Dumbledore 
significant person: Gellert Grindelwald 
significant person: Ariana Dumbledore 
significant person: Harry Potter 
occupation: professor 
occupation: auror 
occupation: researcher 
occupation: school teacher 
employer: Hogwarts 
position held: Headmaster of Hogwarts 
position held: Hogwarts prefect 
position held: Supreme Mugwump 
residence: Mould-on-the-Wold 
member of: Order of the Phoenix 
member of: Gryff

### 1.4 Base de datos vectorial de embeddings

Ahora que ya tengo los PDFs en strings es hora de hacer la base de datos vectorial con los embeddings. Para la base de datos se utilicé ChromaDB y para los embeddings SentenceTransformer.

#### 1.4.1 Split de textos

Aquí utilicé RecursiveCharacterTextSplitter de langchain para dividir los textos. El tamaño del chunk se corresponde a la lógica de que al ser un libro de narración fantástica se necesitan muchos caracteres para obtener el contexto.

In [19]:
# Definir el tamaño y solapamiento de los fragmentos
chunk_size = 1000
chunk_overlap = 200

# Crear el objeto text_splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    is_separator_regex=False
)

# Aplicar la división a los textos de los PDFs
split_texts = text_splitter.create_documents(cleaned_texts)

# Mostrar algunos ejemplos de los fragmentos divididos
for i in range(min(3, len(split_texts))):
    print(f"Fragmento {i + 1}:\n{split_texts[i].page_content}\n")


Fragmento 1:
P a g e  | 2 Harry Potter and the Half Blood Prince – J.K. Rowling

Fragmento 2:
THE OTHER MINISTER  It was nearing midnight and the Prime Minister was  sitting alone in his office, reading a long memo that  was slipping through his brain without leaving the  slightest trace of meaning behind. He was waiting for  a call from the President of a far distant country, and  between wondering when the wretched man would  telephone, and trying to suppress unpleasant  memories of what had been a very long, tiring, and  difficult week, there was not much space in his head  for anything else. The more he attempted to focus on  the print on the page before him, the more clearly the  Prime Minister could see the gloating face of one of  his political opponents. This particular opponent had  appeared on the news that very day, not only to  enumerate all the terrible things that had happened  in the last week (as though anyone needed reminding)  but also to explain why each and every on

#### 1.4.2. Limpieza
Para mejorar la calidad de los embeddings eliminé los símbolos, stop words y lematicé los chunks de texto.

In [20]:
nltk.download('stopwords')
nltk.download('wordnet')

def clean_text_english(text):

    # Convertir a minúsculas
    text = text.lower()

    # Eliminar puntuación
    text = re.sub(r'[^\w\s]', '', text)

    # Eliminar números
    text = re.sub(r'\d', '', text)

    # Conservar símbolos especiales relevantes (agrega otros si es necesario)
    text = re.sub(r"[^a-zA-Z0-9À-ž\s]", '', text)

    # Eliminar stop words en inglés
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    text = ' '.join(filtered_words)

    # Aplicar lematización
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(lemmatized_words)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [21]:
# Aplicar la limpieza a los documentos divididos
cleaned_documents_english = []

for document in split_texts:
    document.page_content = clean_text_english(document.page_content)
    cleaned_documents_english.append(document)

# Mostrar el texto limpio de un ejemplo de documento
print(cleaned_documents_english[1].page_content[:])

minister nearing midnight prime minister sitting alone office reading long memo slipping brain without leaving slightest trace meaning behind waiting call president far distant country wondering wretched man would telephone trying suppress unpleasant memory long tiring difficult week much space head anything else attempted focus print page clearly prime minister could see gloating face one political opponent particular opponent appeared news day enumerate terrible thing happened last week though anyone needed reminding also explain every one government fault prime minister pulse quickened


#### 1.4.3. Embeddings

Una vez que se tienen los chunks limpos se puede llamar a Sentence Transformer para vectorizarlos.

In [22]:
# Cargar el modelo de embeddings
model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
embed_model = SentenceTransformer(model_name)

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.10k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
# Obtener embeddings para cada documento limpio en inglés
embedding_vectors = [embed_model.encode(doc.page_content) for doc in cleaned_documents_english]

# Mostrar los primeros 5 números del primer embedding como ejemplo
print("Embedding del primer documento:")
print(embedding_vectors[0])

##### 1.4.3.1 Exportar e importar embeddings

El proceso anterior tarda una cantidad considerable de tiempo por lo que una vez terminado el proceso, exporté el resultado en CSV para importarlo a convenciencia.

Exportar


In [ ]:
# Convertir a DataFrame
df_embeddings = pd.DataFrame(embedding_vectors)

# Guardar en un archivo CSV
df_embeddings.to_csv('embeddings.csv', index=False)

Importar

In [147]:
# Subir el archivo desde tu computadora
uploaded = files.upload()

# Obtener el nombre del archivo cargado
file_name = list(uploaded.keys())[0]

# Cargar el CSV como DataFrame
df_uploaded = pd.read_csv(file_name)

# Convertir a lista de listas
embedding_vectors = df_uploaded.values.tolist()

#### 1.4.4. Almacenar los embeddings en una base de datos Chromadb

Una vez obtenidos los embeddings de los fragmentos de texto, los almaceno en una base de datos vectorial ChromaDB.

In [149]:
# Obtener el cliente Chroma
chroma_client = chromadb.Client()

# Crear la colección
collection = chroma_client.create_collection(name="HP_embeddings")

# Transformar los chunks sin limpar de una lista de objetos de langchain a una lista de str
documents_str = [langchain_doc_obj.page_content for langchain_doc_obj in split_texts]

# Crear IDs
ids = [f'id{i+1}' for i in range(len(documents_str))]

# Agregar documentos a la colección
collection.add(
    embeddings = embedding_vectors,
    documents = documents_str,
    ids = ids
)

Ejemplo de realizar una query a la base de datos. Primero hay que obtener el embedding de la query y la base buscará los vectores más cercanos, y en este caso, traerá los 5 más cercanos.

In [152]:
# Example query
query_texts = "Who is Albus Dumbledore?"
query_embedding = embed_model.encode(query_texts).tolist()

results = collection.query(query_embeddings= [query_embedding], n_results=5)
for result in results['documents'][0]:
    print(result)
    print('\n')

albus dumbledore greatest headmaster hogwarts ever dobby know sir dobby heard dumbledores power rival whomustnotbenamed height strength sir dobbys voice dropped urgent whisper power dumbledore doesnt power decent wizard harry could stop dobby bounded bed seized harrys desk lamp started beating around head earsplitting yelp sudden silence fell downstairs two second later harry heart thudding madly heard uncle vernonp g e harry potter chamber secret jk rowling coming hall calling dudley must left television little tyke quick closet hissed harry stuffing dobby shutting door flinging onto bed door handle turned


leg elf else simply attempting escape swallowed oncoming horde yet harry sped dueler past strug gling prisoner great hall voldemort center battle striking smiting within reach harry could get clear shot fought way nearer still invisible great hall became crowded everyone could walk forced way inside harry saw yaxley slammed oor george lee jordan saw dolohov fall scream flitwicks h

## 2. Implementación del RAG

### 2.1. Traducción de prompt y respuestas

El enunciado pide que la interacción con el chatbot debe ser en Español pero todas mis fuentes de datos están en inglés y en general los modelos suelen funcionar mejor en inglés. Por lo anterior, opté por traducir la pregunta de entrada al inglés, trabajar en este idioma y volver a traducir al español la respuesta final. Para lograr esto utilicé los modelos "opus-mt-es-en" y "opus-mt-es-en" de Helsinki-NLP disponible en HuggingFace.

#### 2.1.1 - Español a Inglés

In [165]:
def translate_sentence(sentence, model_name="Helsinki-NLP/opus-mt-es-en"):
    # Cargar el tokenizador y el modelo
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Preprocesar la frase de entrada
    input_text = sentence
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Realizar la traducción
    translation_ids = model.generate(input_ids, max_length=1000, num_beams=4, length_penalty=2.0)

    # Decodificar la traducción
    translation = tokenizer.decode(translation_ids[0], skip_special_tokens=True)
    return translation

In [115]:
# Ejemplo de uso
spanish_sentence = "Hola, ¿cómo estás?"
english_translation = translate_sentence(spanish_sentence)

print(f"Frase en español: {spanish_sentence}")
print(f"Traducción al inglés: {english_translation}")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Frase en español: Hola, ¿cómo estás?
Traducción al inglés: Hey, how are you?


#### 2.1.2. Inglés a Español

In [164]:
def translate_sentence_to_spanish(sentence, model_name="Helsinki-NLP/opus-mt-en-es"):
    # Cargar el tokenizador y el modelo
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)

    # Preprocesar la frase de entrada
    input_text = sentence
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Realizar la traducción
    translation_ids = model.generate(input_ids, max_length=1000, num_beams=4, length_penalty=2.0)

    # Decodificar la traducción
    translation = tokenizer.decode(translation_ids[0], skip_special_tokens=True)
    return translation

In [117]:
# Ejemplo de uso
english_sentence = "Hello, how are you?"
spanish_translation = translate_sentence_to_spanish(english_sentence)

print(f"English Sentence: {english_sentence}")
print(f"Spanish Translation: {spanish_translation}")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

English Sentence: Hello, how are you?
Spanish Translation: Hola, ¿cómo estás?


### 2.2. Clasificación para elegir fuente de contexto
Para clasificar las preguntas y elegir la fuente para dar contexto, utilicé un LLM específicamente "zephyr-7b-beta" diponible en Hugging face y un formato de prompt Few-Shot para obtener respuestas más exactas.

In [107]:
def zephyr_chat_template(messages, add_generation_prompt=True):
    # Definir la plantilla Jinja
    template_str  = "{% for message in messages %}"
    template_str += "{% if message['role'] == 'user' %}"
    template_str += "<|user|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'assistant' %}"
    template_str += "<|assistant|>{{ message['content'] }}</s>\n"
    template_str += "{% elif message['role'] == 'system' %}"
    template_str += "<|system|>{{ message['content'] }}</s>\n"
    template_str += "{% else %}"
    template_str += "<|unknown|>{{ message['content'] }}</s>\n"
    template_str += "{% endif %}"
    template_str += "{% endfor %}"
    template_str += "{% if add_generation_prompt %}"
    template_str += "<|assistant|>\n"
    template_str += "{% endif %}"

    # Crear un objeto de plantilla con la cadena de plantilla
    template = Template(template_str)

    # Renderizar la plantilla con los mensajes proporcionados
    return template.render(messages=messages, add_generation_prompt=add_generation_prompt)



# Aquí hacemos la llamada el modelo
def choose_data_source(prompt: str, api_key, max_new_tokens: int = 768) -> None:
    messages: List[Dict[str, str]] = [
                                {
                                  "role": "system",
                                  "content": "You are a bibliographic assistant, specialized in identifying the subject or object referred to in a question or task."
                                },
                                {
                                  "role": "user",
                                  "content": "What is the full name of Albus Dumbledore?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Personaje: [Albus Dumbledore]"
                                },
                                {
                                  "role": "user",
                                  "content": "Who are the siblings of Ron Weasley?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Personaje: [Ron Weasley]"
                                },
                                {
                                  "role": "user",
                                  "content": "Which house is Luna Lovegood in?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Personaje: [Luna Lovegood]"
                                },
                                {
                                  "role": "user",
                                  "content": "In what year was the Goblet of Fire released?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Nobelas"
                                },
                                {
                                  "role": "user",
                                  "content": "How much the Deathly Hallows book raised?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "Novelas"
                                },
                                {
                                  "role": "user",
                                  "content": "What form does the Boggart take in front of Neville?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "History"
                                },
                                {
                                  "role": "user",
                                  "content": "What were Snape's last words?"
                                },
                                {
                                  "role": "assistant",
                                  "content": "History"
                                },
                                {
                                  "role": "user",
                                  "content": f'Respond only in one of the following ways without using prior knowledge of the topic or adding information not contained in the question:\n\
                                  "Personaje: [Name of the explicitly mentioned character in the question]" if it is a question or task about a character, with the name capitalized.\n\
                                  "Novelas" if it is a question or task about the physical books of Harry Potter.\n\
                                  "Historia" if it is about the story told in the Harry Potter novels.\n\
                                  -------------------------------------------\n\
                                  The question or task is as follows:\n\
                                  {prompt}'
                                }
                              ]


    try:
        prompt_formatted: str = zephyr_chat_template(messages, add_generation_prompt=True)

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {api_key}"}

        # Datos para enviar en la solicitud POST
				# Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
        data = {
            "inputs": prompt_formatted,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizamos la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Extraer respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt_formatted):]
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")

In [139]:
# Ejemplo)
prompt = 'Where does Minerva MacGonagall work?'
answer = choose_data_source(prompt, api_key=api_key)
print('\nRESPUESTA: \n', answer)


RESPUESTA: 
 "Personaje: [Minerva McGonagall]" for questions or tasks about Minerva McGonagall's identity, such as where she works.

For the given question, the answer would be: "Personaje: [Minerva McGonagall]"


### 2.3. Interpretar respuesta

Ya que le pedí un formato de respuesta muy específico al LLM, puedo utilizar una función muy sencilla para obtener la información que necesito. La información la formatearé en una lista con dos elementos. El primero con el tema de la pregunta ("Personaje", "Novelas", "Historia") y el segundo en None a menos que sea una pregunta sobre un personaje, dónde también obtendré el personaje en cuestión para realizar la query en la base de datos de grafos.

In [134]:
def answer_cleaner(texto):
    palabras_clave = ["Personaje", "Novelas", "Historia"]

    for palabra in palabras_clave:
        if palabra in texto:
            if palabra == "Personaje":
                match = re.search(r'\[([^]]*)\]', texto)
                if match:
                    texto_entre_corchetes = match.group(1)
                    return [palabra, texto_entre_corchetes]
            else:
                return [palabra, None]

    return [None, None]

# Ejemplo de uso:
respuesta_limpia = answer_cleaner(answer)
print(respuesta_limpia)


['Personaje', 'Minerva McGonagall']


### 2.4. Devolver contexto

Ahora teniendo el resultado de la clasificación, le paso la información necesaria a cada fuente para obtener el contexto.

In [135]:
def devolver_contexto(respuesta_LLM, prompt):
    # Si no se halló nada, no devolver contexto
    if respuesta_LLM in ([None, None], ["Personaje", None]):
        return ("", 'Error al identificar la fuente')

    # Si la pregunta o tarea es referida a un personaje
    elif respuesta_LLM[0] == "Personaje":
        propiedades_personaje = obtener_valores_por_sujeto(respuesta_LLM[1])
        return (propiedades_personaje, 'Base de datos de grafos')

    # Si la pregunta o tarea es referida a las nobelas
    elif respuesta_LLM[0] == "Novelas":
        dataframe_string = dataframe_to_string(df_harry_potter)
        return (dataframe_string, 'Dataframe')

    # Si la pregunta o tarea es referida a la historia
    elif respuesta_LLM[0] == "Historia":
        contexto = ''
        # Obtener el cliente Chroma
        chroma_client = chromadb.Client()
        # Obtener la colección
        collection = chroma_client.get_collection(name="HP_embeddings")
        # Query
        query_embedding = embed_model.encode(prompt).tolist()
        results = collection.query(query_embeddings= [query_embedding], n_results=5)
        for result in results['documents'][0]:
          contexto += result

        return (contexto, "Base de datos vectorial de Embeddings")

In [136]:
# Ejemplo
contexto = devolver_contexto(respuesta_limpia, prompt)[0]

print(contexto)

Minerva McGonagall  :
sex or gender: female 
country of citizenship: United Kingdom 
name in native language: Minerva McGonagall 
given name: Minerva 
date of birth: 1935-10-04T00:00:00Z 
occupation: professor 
occupation: employee 
occupation: head teacher 
occupation: school teacher 
employer: Hogwarts 
employer: Department of Magical Law Enforcement 
position held: Headmaster of Hogwarts 
member of: Order of the Phoenix 
member of: Gryffindor 



### 2.5. Preguntar al agente con contexto

El último paso es preguntarle al LLM que nuevamente es el "zephyr-7b-beta" pero añadiendo a la pregunta el contexto y pidiéndole que no utilice su conocimiento previo.

In [140]:
def ask_agent_context(prompt: str, context: str, api_key, max_new_tokens: int = 768) -> None:
    messages: List[Dict[str, str]] = [
                                    {
                                        "role": "system",
                                        "content": "You are a helpful assistant that always responds with truthful, useful, and fact-based answers."
                                    },
                                    {
                                        "role": "user",
                                        "content": f"The context information is as follows:\n\
                                        ---------------------\n\
                                        {context}\n\
                                        ---------------------\n\
                                        Given the above context information, and without using prior knowledge, answer the following question.\n\
                                        Question: {prompt}\n\
                                        Answer: "
                                    }
                                ]

    try:
        prompt_formatted: str = zephyr_chat_template(messages, add_generation_prompt=True)

        # URL de la API de Hugging Face para la generación de texto
        api_url = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"

        # Cabeceras para la solicitud
        headers = {"Authorization": f"Bearer {api_key}"}

        # Datos para enviar en la solicitud POST
				# Sobre los parámetros: https://huggingface.co/docs/transformers/main_classes/text_generation
        data = {
            "inputs": prompt_formatted,
            "parameters": {
                "max_new_tokens": max_new_tokens,
                "temperature": 0.7,
                "top_k": 50,
                "top_p": 0.95
            }
        }

        # Realizamos la solicitud POST
        response = requests.post(api_url, headers=headers, json=data)

        # Extraer respuesta
        respuesta = response.json()[0]["generated_text"][len(prompt_formatted):]
        return respuesta

    except Exception as e:
        print(f"An error occurred: {e}")

In [143]:
#Ejemplo
respuesta_final = ask_agent_context(prompt, contexto, api_key=api_key)

print(f'Prompt: {prompt}\n')
print('-----------------------------------------------------------------------')
print(f'Contexto: {contexto}\n')
print('-----------------------------------------------------------------------')
print(f'Respuesta:\n {respuesta_final}')

Prompt: Cuéntame algo que haya pasado en el Gran Comedor de Hogwarts

-----------------------------------------------------------------------
Contexto: toilet potter boy hearing theyve changed time venu e start eigh oclock old courtroom ten old told merlin beard mr weasley looked watch let yelp leapt chair quick harry five minute ago perkins flattened th e filing cabinet mr weasley left office ru n harry heel changed time harry said breath lessly hurtled past auror cubicle people poked head stared streaked past harry felt though left side back perkinss desk ive idea thank goodness got early youd missed would catastrophic mr weasley skidded halt side lift jabbed impa tiently buttonoh well said slowly let see chamber secret know course hogwarts founded thousand year ago precise date uncertain four greatest witch wizard age four school house named godric gryffindor helga hufflepuff rowena ravenclaw salazar slytherin built castle together far prying muggle eye age magic feared common peopl

##3. RESULTADO FINAL

### 3.1. Función aglomerativa

Creo una función para utilizar más cómodamente RAG implementado.

In [168]:
def RAG(prompt, api_key):
    #Traducir el prompt al inglés
    english_prompt = translate_sentence(prompt)

    # Identificar la fuente
    source_raw = choose_data_source(english_prompt , api_key=api_key)

    # Formatear respuesta del LLM para la fuente
    source_clean = answer_cleaner(source_raw)

    # Obtener el contexto de la fuente indicada y el ID de la fuente
    contexto, id_source = devolver_contexto(source_clean, prompt)

    # Obtener la respuesta con el prompt original más el contexto obtenido de las fuentes externas
    respuesta_final = ask_agent_context(prompt, contexto, api_key=api_key)

    # Traducir la respuesta al Español
    respuesta_final_español = translate_sentence_to_spanish(respuesta_final)

    return (respuesta_final_español, id_source, contexto)

### 3.2. Ejemplos de uso del RAG

#### Ejemplo 1

In [159]:
# Prompt:
prompt = '¿Cuáles son los hermanos de Ron Weasley?'

# Llamar a la función del RAG
respuesta, fuente, contexto = RAG(prompt, api_key)

# Imprimir el resultado
print('------------------------------------------------------------------------')
print(f'PREGUNTA:\n{prompt}')
print('------------------------------------------------------------------------')
print(f'FUENTE ESCOGIDA: {fuente}')
print('------------------------------------------------------------------------')
print(f'CONTEXTO BRINDADO:\n{contexto}')
print('------------------------------------------------------------------------')
print(f'RESPUESTA:\n{respuesta}')

------------------------------------------------------------------------
PREGUNTA:
¿Cuáles son los hermanos de Ron Weasley?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos de grafos
------------------------------------------------------------------------
CONTEXTO BRINDADO:
Ron Weasley  :
sex or gender: male 
country of citizenship: United Kingdom 
name in native language: Ron Weasley 
given name: Ron 
given name: Ronald 
date of birth: 1980-03-01T00:00:00Z 
place of birth: Devon 
father: Arthur Weasley 
mother: Molly Weasley 
sibling: Ginny Weasley 
sibling: Bill Weasley 
sibling: Percy Weasley 
sibling: Charlie Weasley 
sibling: Fred Weasley 
sibling: George Weasley 
occupation: student 
occupation: entrepreneur 
occupation: magician 
occupation: auror 
occupation: Quidditch player 
employer: Weasleys' Wizard Wheezes 
position held: Hogwarts prefect 
residence: The Burrow 
residence: Hogwarts Castle 
member of: Dumbledore's Army 

#### Ejemplo 2

In [169]:
# Prompt:
prompt = 'Qué pasó en la cámara de los secretos?'

# Llamar a la función del RAG
respuesta, fuente, contexto = RAG(prompt, api_key)

# Imprimir el resultado
print('------------------------------------------------------------------------')
print(f'PREGUNTA:\n{prompt}')
print('------------------------------------------------------------------------')
print(f'FUENTE ESCOGIDA: {fuente}')
print('------------------------------------------------------------------------')
print(f'CONTEXTO BRINDADO:\n{contexto}')
print('------------------------------------------------------------------------')
print(f'RESPUESTA:\n{respuesta}')

------------------------------------------------------------------------
PREGUNTA:
Qué pasó en la cámara de los secretos?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Base de datos vectorial de Embeddings
------------------------------------------------------------------------
CONTEXTO BRINDADO:
nooooooo scene whirled darkness became complete harry felt falling crash landed spreadeagled fourposter gryffindor dormitory riddle diary lying open stomach time regain breath dormitory door opened ron came said harry sat sweating shaking whats said ron looking concern hagrid ron hagrid opened chamber secret fifty year agop g e harry potter chamber secret j k rowlingone nighttime stroll office potter pay right said harry coolly turning back ginger root ill bear mind ever get urge go snapes eye flashed plunged hand inside black robe one wild mo ment harry thought snape pull wand curse saw snape drawn small crystal bottle completely clear po tion harry

#### Ejemplo 3

In [157]:
# Prompt:
prompt = 'Cuántas unidades se vendieron de "La cámara secreta"?'

# Llamar a la función del RAG
respuesta, fuente, contexto = RAG(prompt, api_key)

# Imprimir el resultado
print('------------------------------------------------------------------------')
print(f'PREGUNTA:\n{prompt}')
print('------------------------------------------------------------------------')
print(f'FUENTE ESCOGIDA: {fuente}')
print('------------------------------------------------------------------------')
print(f'CONTEXTO BRINDADO:\n{contexto}')
print('------------------------------------------------------------------------')
print(f'RESPUESTA:\n{respuesta}')

------------------------------------------------------------------------
PREGUNTA:
Cuántas unidades se vendieron de "La cámara secreta"?
------------------------------------------------------------------------
FUENTE ESCOGIDA: Dataframe
------------------------------------------------------------------------
CONTEXTO BRINDADO:
Book Name: Harry Potter and the Sorcerer’s Stone
Copies Sold in UK: 4.2 millons
Copies Sold WorldWide: 120 millons
Publish Date: 26 June 1997
Pages: 223
Words: 76,944
Audiobook: 9 hrs and 33 mins
Total Awards Won: 8

Book Name: Harry Potter and the Chamber of Secrets
Copies Sold in UK: 3.5 millons
Copies Sold WorldWide: 77 millons
Publish Date: 2 July 1998
Pages: 251
Words: 85,141
Audiobook: 11 hrs and 5 mins
Total Awards Won: 5

Book Name: Harry Potter and the Prisoner of Azkaban
Copies Sold in UK: 3.4 millons
Copies Sold WorldWide: 65 millons
Publish Date: 8 July 1999
Pages: 317
Words: 107,253
Audiobook: 13 hrs and 10 mins
Total Awards Won: 3

Book Name: Harry 

## 4. HAZ TU PREGUNTA SOBRE HARRY POTTER

In [ ]:
# Prompt:
prompt = input("Prompt: ")

# Llamar a la función del RAG
respuesta, fuente, contexto = RAG(prompt, api_key)

# Imprimir el resultado
print('------------------------------------------------------------------------')
print(f'PREGUNTA:\n{prompt}')
print('------------------------------------------------------------------------')
print(f'FUENTE ESCOGIDA: {fuente}')
print('------------------------------------------------------------------------')
print(f'CONTEXTO BRINDADO:\n{contexto}')
print('------------------------------------------------------------------------')
print(f'RESPUESTA:\n{respuesta}')